![alt text](imgs/sonosco_3.jpg "Titel")

## Data

In [11]:
import IPython.display as ipd
import os

file_id = "1995-1826-0007"

audio_file = os.path.expanduser(f"~/temp/data/libri_speech/test_clean/wav/{file_id}.wav")
transcript_file = os.path.expanduser(f"~/temp/data/libri_speech/test_clean/txt/{file_id}.txt")

ipd.Audio(audio_file)

In [12]:
with open(transcript_file, 'r') as f:
    print(f'Transcription: {f.read()}')

Transcription: FIND SOME CRESSWELLS THERE BIG PLANTATIONS RATED AT TWO HUNDRED AND FIFTY THOUSAND DOLLARS


In [13]:
import logging
import sonosco

from sonosco.common.constants import SONOSCO
from sonosco.common.utils import setup_logging
from importlib import reload

reload(logging)

logging.basicConfig(level=logging.INFO)

In [14]:
from sonosco.datasets.download_datasets.create_manifest import create_manifest

In [15]:
path_to_data = '/Users/yuriy/Documents/University/ss2019/sonosco/demo/data/'
output_file = '/Users/yuriy/Documents/University/ss2019/sonosco/demo/data/manifest.csv'

create_manifest(path_to_data, output_file)

INFO:sonosco.datasets.download_datasets.create_manifest:Creating a manifest for path: /Users/yuriy/Documents/University/ss2019/sonosco/demo/data/
INFO:sonosco.datasets.download_datasets.create_manifest:Found 1 .wav files
INFO:sonosco.datasets.download_datasets.create_manifest:Sorting manifests...
100%|██████████| 1/1 [00:00<00:00, 35.10it/s]


In [16]:
import pandas as pd

logging.info("Manifest:")

pd.read_csv('./data/manifest.csv', names=["Audio", "Text", "Duration"])

INFO:root:Manifest:


,Audio,Text,Duration
0,/Users/yuriy/Documents/University/ss2019/sonos...,/Users/yuriy/Documents/University/ss2019/sonos...,10.435


# Training

In [17]:
import torch

from sonosco.training.experiment import Experiment
from sonosco.training.trainer import ModelTrainer
from sonosco.model.deserializer import Deserializer
from sonosco.decoders import GreedyDecoder

from sonosco.models.seq2seq_las import Seq2Seq

from sonosco.common.path_utils import parse_yaml
from sonosco.datasets import create_data_loaders

from sonosco.training.word_error_rate import word_error_rate
from sonosco.training.character_error_rate import character_error_rate
from sonosco.training.losses import cross_entropy_loss

In [8]:
config = parse_yaml('./train_seq2seq_demo.yaml')['train']
device = torch.device("cpu")

train_loader, val_loader, test_loader = create_data_loaders(**config)

INFO:sonosco.datasets.loader:Training dataset containing 2620 samples is created
INFO:sonosco.datasets.loader:Training data loader created.
INFO:sonosco.datasets.loader:Validation dataset containing 32 samples is created
INFO:sonosco.datasets.loader:Validation data loader created.
INFO:sonosco.datasets.loader:Test dataset containing 2620 samples is created
INFO:sonosco.datasets.loader:Test data loader created.


In [18]:
kwargs = {
    'loss': cross_entropy_loss,
    'epochs': config["max_epochs"],
    'train_data_loader': train_loader,
    'val_data_loader': val_loader,
    'test_data_loader': test_loader,
    'lr': config["learning_rate"],
    'custom_model_eval': True,
    'metrics': [word_error_rate, character_error_rate],
    'decoder': GreedyDecoder(config['labels']),
    'device': device, 
    'test_step': config["test_step"]
}

In [19]:
experiment = Experiment.create(config, logging.getLogger())

CONTINUE = False

if CONTINUE:
    loader = Deserializer()
    trainer: ModelTrainer = loader.deserialize(ModelTrainer, config["checkpoint_path"], {
            'train_data_loader': train_loader,
            'val_data_loader': val_loader,
            'test_data_loader': test_loader,
    }, with_config=True)
else:
    model = Seq2Seq(config["encoder"], config["decoder"])
    trainer = ModelTrainer(model, **kwargs)

experiment.setup_model_trainer(trainer, checkpoints=True, tensorboard=True)

try:
    experiment.start()
except KeyboardInterrupt:
    experiment.stop()

INFO:sonosco.training.trainer:Compute Metrics
INFO:sonosco.training.trainer:Compute metric : word_error_rate
INFO:sonosco.training.trainer:Compute metric : character_error_rate
INFO:sonosco.training.trainer:epoch 1   batch 0/654   character_error_rate: 88.729897   loss: 3.333069   word_error_rate: 100.000000
INFO:sonosco.training.tensorboard_callback:performance measure: word_error_rate, 100.0
INFO:sonosco.training.tensorboard_callback:performance measure: character_error_rate, 88.7298965002317
INFO:sonosco.training.tensorboard_callback:performance measure: gradient_norm, 0.3435996472835541
INFO:sonosco.training.tensorboard_callback:performance measure: loss, 3.333069086074829
INFO:sonosco.training.trainer:Compute Metrics
INFO:sonosco.training.trainer:Compute metric : word_error_rate
INFO:sonosco.training.trainer:Compute metric : character_error_rate
INFO:sonosco.training.trainer:epoch 1   batch 1/654   character_error_rate: 87.742929   loss: 3.332753   word_error_rate: 100.000000
INFO